Similar to Experiment 75, but here we use the version of SMC-RWM and MS-RWM that is in the python script. In particular:

- Weights are computed and normalized using the log-sum-exp trick.
- Sequence of tolerances is chosen adaptively based on quantile of distances.
- Particles are initially sampled from the prior but then we set $\epsilon_0 = \max d_{i}$ where $d_i$ are the distances between the particle $i$ and the manifold.
- Step size is adapted based on the proxy-acceptance-probability

In [1]:
import numpy as np
from numpy import zeros, eye, array, diag, exp, ones
from numpy.linalg import solve, norm
from numpy.random import choice, default_rng
from scipy.stats import multivariate_normal as MVN
from scipy.special import logsumexp
from scipy.stats import uniform as udist
import math
import time
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib import rc
from ipywidgets.widgets import IntSlider
from ipywidgets.widgets import interact
from warnings import resetwarnings

from Manifolds.GeneralizedEllipseNew import GeneralizedEllipse
from markov_snippets import MSAdaptive, SMCAdaptive

In [2]:
def generate_settings(
    d=10, 
    N=5000,
    B=20,
    δ=0.1,
    δmin=1e-3,
    δmax=100.0,
    εmin=1e-16,
    min_pm=1e-4,
    maxiter=500,
    verbose=True,
    εs_fixed=None,
    adaptiveε=True,
    adaptiveδ=True,
    z0_manual=None,
    pm_target=0.23,
    pm_switch=0.05,
    prior_seed=1234,
    low_memory=True,
    integrator='rwm',
    εprop_switch=0.01,
    ε0_manual=None,
    quantile_value=0.9,
    initialization='prior',
    switch_strategy='pm',
    prior='uniform',
    kernel='uniform',
    prior_scale=7,
    thinning=10,
    burnin=100):
    # Generate the manifold
    μ = zeros(d)
    Σ = diag(np.r_[0.1, ones(d-1)])  
    level_set_value = -23
    manifold = GeneralizedEllipse(μ, Σ, exp(level_set_value), prior=prior, kernel=kernel, prior_scale=prior_scale)
    # grab arguments
    arguments = locals()
    return arguments

In [3]:
SETTINGS_RWM           = generate_settings(integrator='rwm')
SETTINGS_THUG          = generate_settings(integrator='thug')
SETTINGS_RWM_THEN_THUG = generate_settings(integrator='rwm_then_thug')

# Random Walk Only

#### SMC

In [4]:
SMC_RWM = SMCAdaptive(SETTINGS_RWM)
_ = SMC_RWM.sample()

Stochastic Kernel: RWM.
Setting initial epsilon to εmax = 359.0601375831
Iteration:  1
	Particles resampled.
	Epsilon: 258.6657430509
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.38060000
	Step-size adapted to: 0.10782076
Iteration:  2
	Particles resampled.
	Epsilon: 219.2638857537
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.38000000
	Step-size adapted to: 0.11621828
Iteration:  3
	Particles resampled.
	Epsilon: 186.2348569565
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.36260000
	Step-size adapted to: 0.12418473
Iteration:  4
	Particles resampled.
	Epsilon: 159.1554746308
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.34320000
	Step-size adapted to: 0.13141631
Iteration:  5
	Particles resampled.
	Epsilon: 137.1687542986
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.33660000
	Step-size adapted to: 0.13861082
Iteration:  6
	Particles resampled.
	Epsilon: 121.2658824421
	

	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.17980000
	Step-size adapted to: 0.11625316
Iteration:  49
	Particles resampled.
	Epsilon: 14.5910355185
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.18860000
	Step-size adapted to: 0.11387145
Iteration:  50
	Particles resampled.
	Epsilon: 13.9056796918
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.18380000
	Step-size adapted to: 0.11127117
Iteration:  51
	Particles resampled.
	Epsilon: 13.0843062618
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.19100000
	Step-size adapted to: 0.10912240
Iteration:  52
	Particles resampled.
	Epsilon: 12.3257975179
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.17700000
	Step-size adapted to: 0.10626864
Iteration:  53
	Particles resampled.
	Epsilon: 11.5514862613
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.16560000
	Step-size adapted to: 0.10290129
Iteration:  54
	Particl

	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.03780000
	Step-size adapted to: 0.00850854
Iteration:  97
	Particles resampled.
	Epsilon: 0.1615912075
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.03720000
	Step-size adapted to: 0.00772661
Iteration:  98
	Particles resampled.
	Epsilon: 0.1401333790
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.03220000
	Step-size adapted to: 0.00699902
Iteration:  99
	Particles resampled.
	Epsilon: 0.1260298617
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.03100000
	Step-size adapted to: 0.00633614
Iteration:  100
	Particles resampled.
	Epsilon: 0.1124426221
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.03520000
	Step-size adapted to: 0.00574810
Iteration:  101
	Particles resampled.
	Epsilon: 0.1009842422
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.02780000
	Step-size adapted to: 0.00519538
Iteration:  102
	Particles

	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.00080000
	Step-size adapted to: 0.00100000
Iteration:  145
	Particles resampled.
	Epsilon: 0.0006398924
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.00040000
	Step-size adapted to: 0.00100000
Iteration:  146
	Particles resampled.
	Epsilon: 0.0005384727
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.00040000
	Step-size adapted to: 0.00100000
Iteration:  147
	Particles resampled.
	Epsilon: 0.0004526726
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.00000000
	Step-size adapted to: 0.00100000


#### Markov Snippets

In [5]:
MS_RWM = MSAdaptive(SETTINGS_RWM)
_ = MS_RWM.sample()

Integrator: RWM.
Setting initial epsilon to εmax = 359.0601375831
Iteration:  1
	Trajectories constructed.
	Epsilon: 257.28439548
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.922
	Step-size adapted to: 0.14135440
Iteration:  2
	Trajectories constructed.
	Epsilon: 211.65529961
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.921
	Step-size adapted to: 0.19967084
Iteration:  3
	Trajectories constructed.
	Epsilon: 175.25494707
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.896
	Step-size adapted to: 0.27854237
Iteration:  4
	Trajectories constructed.
	Epsilon: 145.87633884
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.860
	Step-size adapted to: 0.38171345
Iteration:  5
	Trajectories constructed.
	Epsilon: 124.44512710
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop M

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.175
	Step-size adapted to: 1.40735901
Iteration:  44
	Trajectories constructed.
	Epsilon: 6.78610601
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.180
	Step-size adapted to: 1.37274846
Iteration:  45
	Trajectories constructed.
	Epsilon: 6.17126154
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.184
	Step-size adapted to: 1.34126729
Iteration:  46
	Trajectories constructed.
	Epsilon: 5.56137209
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.192
	Step-size adapted to: 1.31589219
Iteration:  47
	Trajectories constructed.
	Epsilon: 5.02573776
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.186
	Step-size adapted to: 1.28751616
Iteration:  48
	Trajectories constructed.
	Epsilon: 4.45470094
	Weights computed and nor

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.095
	Step-size adapted to: 0.19656048
Iteration:  87
	Trajectories constructed.
	Epsilon: 0.06979822
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.105
	Step-size adapted to: 0.18465149
Iteration:  88
	Trajectories constructed.
	Epsilon: 0.06226720
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.096
	Step-size adapted to: 0.17266792
Iteration:  89
	Trajectories constructed.
	Epsilon: 0.05682674
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.097
	Step-size adapted to: 0.16155897
Iteration:  90
	Trajectories constructed.
	Epsilon: 0.05192041
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.097
	Step-size adapted to: 0.15119497
Iteration:  91
	Trajectories constructed.
	Epsilon: 0.04702101
	Weights computed and nor

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.060
	Step-size adapted to: 0.00765279
Iteration:  129
	Trajectories constructed.
	Epsilon: 0.00081985
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.053
	Step-size adapted to: 0.00700392
Iteration:  130
	Trajectories constructed.
	Epsilon: 0.00073954
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.051
	Step-size adapted to: 0.00640494
Iteration:  131
	Trajectories constructed.
	Epsilon: 0.00067060
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.050
	Step-size adapted to: 0.00585251
Iteration:  132
	Trajectories constructed.
	Epsilon: 0.00060311
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.052
	Step-size adapted to: 0.00535414
Iteration:  133
	Trajectories constructed.
	Epsilon: 0.00055345
	Weights computed an

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.006
	Step-size adapted to: 0.00100000
Iteration:  171
	Trajectories constructed.
	Epsilon: 0.00000913
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.003
	Step-size adapted to: 0.00100000
Iteration:  172
	Trajectories constructed.
	Epsilon: 0.00000874
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.005
	Step-size adapted to: 0.00100000
Iteration:  173
	Trajectories constructed.
	Epsilon: 0.00000799
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.005
	Step-size adapted to: 0.00100000
Iteration:  174
	Trajectories constructed.
	Epsilon: 0.00000692
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.004
	Step-size adapted to: 0.00100000
Iteration:  175
	Trajectories constructed.
	Epsilon: 0.00000623
	Weights computed an

# Random Walk switch to THUG

#### SMC

In [6]:
SMC_RWM_THEN_THUG = SMCAdaptive(SETTINGS_RWM_THEN_THUG)
_ = SMC_RWM_THEN_THUG.sample()

Stochastic Kernel: RWM.
Setting initial epsilon to εmax = 359.0601375831
Iteration:  1
	Particles resampled.
	Epsilon: 257.5428362993
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.40400000
	Step-size adapted to: 0.10908967
Iteration:  2
	Particles resampled.
	Epsilon: 218.6535839953
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.36900000
	Step-size adapted to: 0.11694108
Iteration:  3
	Particles resampled.
	Epsilon: 186.0919481729
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.36480000
	Step-size adapted to: 0.12509459
Iteration:  4
	Particles resampled.
	Epsilon: 159.6156868651
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.34700000
	Step-size adapted to: 0.13263091
Iteration:  5
	Particles resampled.
	Epsilon: 139.4824590622
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.33180000
	Step-size adapted to: 0.13955659
Iteration:  6
	Particles resampled.
	Epsilon: 122.4486541481
	

	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.17800000
	Step-size adapted to: 0.11585857
Iteration:  49
	Particles resampled.
	Epsilon: 13.9929033830
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.18640000
	Step-size adapted to: 0.11336018
Iteration:  50
	Particles resampled.
	Epsilon: 13.2246005694
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.17700000
	Step-size adapted to: 0.11039559
Iteration:  51
	Particles resampled.
	Epsilon: 12.4474232919
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.16720000
	Step-size adapted to: 0.10698303
Iteration:  52
	Particles resampled.
	Epsilon: 11.7345109155
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.17480000
	Step-size adapted to: 0.10407067
Iteration:  53
	Particles resampled.
	Epsilon: 11.0293944728
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.17100000
	Step-size adapted to: 0.10104543
Iteration:  54
	Particl

	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.87980000
	Step-size adapted to: 0.11712833
Iteration:  96
	Particles resampled.
	Epsilon: 0.1818258638
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.86460000
	Step-size adapted to: 0.16086575
Iteration:  97
	Particles resampled.
	Epsilon: 0.1647125400
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.82260000
	Step-size adapted to: 0.21634410
Iteration:  98
	Particles resampled.
	Epsilon: 0.1481777917
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.74800000
	Step-size adapted to: 0.28030273
Iteration:  99
	Particles resampled.
	Epsilon: 0.1340081264
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.64500000
	Step-size adapted to: 0.34493990
Iteration:  100
	Particles resampled.
	Epsilon: 0.1219206072
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.54020000
	Step-size adapted to: 0.40281209
Iteration:  101
	Particles 

	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.14540000
	Step-size adapted to: 0.11972174
Iteration:  144
	Particles resampled.
	Epsilon: 0.0009409591
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.13540000
	Step-size adapted to: 0.11419074
Iteration:  145
	Particles resampled.
	Epsilon: 0.0008322641
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.13600000
	Step-size adapted to: 0.10894794
Iteration:  146
	Particles resampled.
	Epsilon: 0.0007437561
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.13640000
	Step-size adapted to: 0.10396665
Iteration:  147
	Particles resampled.
	Epsilon: 0.0006673978
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.13980000
	Step-size adapted to: 0.09938192
Iteration:  148
	Particles resampled.
	Epsilon: 0.0005881200
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.13980000
	Step-size adapted to: 0.09499936
Iteration:  149
	Partic

	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.14280000
	Step-size adapted to: 0.00889127
Iteration:  192
	Particles resampled.
	Epsilon: 0.0000043266
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.15540000
	Step-size adapted to: 0.00856574
Iteration:  193
	Particles resampled.
	Epsilon: 0.0000038959
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.15580000
	Step-size adapted to: 0.00825377
Iteration:  194
	Particles resampled.
	Epsilon: 0.0000035090
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.16160000
	Step-size adapted to: 0.00797626
Iteration:  195
	Particles resampled.
	Epsilon: 0.0000031095
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.15420000
	Step-size adapted to: 0.00767962
Iteration:  196
	Particles resampled.
	Epsilon: 0.0000027050
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.16420000
	Step-size adapted to: 0.00743107
Iteration:  197
	Partic

	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.15540000
	Step-size adapted to: 0.00152664
Iteration:  240
	Particles resampled.
	Epsilon: 0.0000000206
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.15160000
	Step-size adapted to: 0.00146795
Iteration:  241
	Particles resampled.
	Epsilon: 0.0000000184
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.15480000
	Step-size adapted to: 0.00141378
Iteration:  242
	Particles resampled.
	Epsilon: 0.0000000165
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.15160000
	Step-size adapted to: 0.00135944
Iteration:  243
	Particles resampled.
	Epsilon: 0.0000000147
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.15280000
	Step-size adapted to: 0.00130796
Iteration:  244
	Particles resampled.
	Epsilon: 0.0000000131
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.14240000
	Step-size adapted to: 0.00125191
Iteration:  245
	Partic

	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.00440000
	Step-size adapted to: 0.00100000
Iteration:  288
	Particles resampled.
	Epsilon: 0.0000000001
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.00640000
	Step-size adapted to: 0.00100000
Iteration:  289
	Particles resampled.
	Epsilon: 0.0000000001
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.00400000
	Step-size adapted to: 0.00100000
Iteration:  290
	Particles resampled.
	Epsilon: 0.0000000001
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.00480000
	Step-size adapted to: 0.00100000
Iteration:  291
	Particles resampled.
	Epsilon: 0.0000000001
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.00520000
	Step-size adapted to: 0.00100000
Iteration:  292
	Particles resampled.
	Epsilon: 0.0000000001
	Weights computed and normalised.
	Mutation step done.
	Approx AP: 0.00440000
	Step-size adapted to: 0.00100000
Iteration:  293
	Partic

#### Markov Snippets

In [ ]:
MS_RWM_THEN_THUG = MSAdaptive(SETTINGS_RWM_THEN_THUG)
_ = MS_RWM_THEN_THUG.sample()

Integrator: RWM.
Setting initial epsilon to εmax = 359.0601375831
Iteration:  1
	Trajectories constructed.
	Epsilon: 257.28439548
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.923
	Step-size adapted to: 0.14142509
Iteration:  2
	Trajectories constructed.
	Epsilon: 215.21915524
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.918
	Step-size adapted to: 0.19951116
Iteration:  3
	Trajectories constructed.
	Epsilon: 177.95869065
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.898
	Step-size adapted to: 0.27857023
Iteration:  4
	Trajectories constructed.
	Epsilon: 148.22886466
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.872
	Step-size adapted to: 0.38408742
Iteration:  5
	Trajectories constructed.
	Epsilon: 124.38178110
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop M

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.194
	Step-size adapted to: 1.41483782
Iteration:  44
	Trajectories constructed.
	Epsilon: 6.82824300
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.200
	Step-size adapted to: 1.39377363
Iteration:  45
	Trajectories constructed.
	Epsilon: 6.19522891
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.196
	Step-size adapted to: 1.37014272
Iteration:  46
	Trajectories constructed.
	Epsilon: 5.60409358
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.171
	Step-size adapted to: 1.33031387
Iteration:  47
	Trajectories constructed.
	Epsilon: 5.09517105
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.186
	Step-size adapted to: 1.30123642
Iteration:  48
	Trajectories constructed.
	Epsilon: 4.59366472
	Weights computed and nor

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.100
	Step-size adapted to: 0.20542278
Iteration:  87
	Trajectories constructed.
	Epsilon: 0.07770961
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.108
	Step-size adapted to: 0.19322787
Iteration:  88
	Trajectories constructed.
	Epsilon: 0.07059693
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.099
	Step-size adapted to: 0.18101324
Iteration:  89
	Trajectories constructed.
	Epsilon: 0.06409031
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.103
	Step-size adapted to: 0.16985926
Iteration:  90
	Trajectories constructed.
	Epsilon: 0.05741839
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.108
	Step-size adapted to: 0.15979156
Iteration:  91
	Trajectories constructed.
	Epsilon: 0.05139168
	Weights computed and nor

	Trajectories constructed.
	Epsilon: 0.00103301
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.951
	Step-size adapted to: 0.01328945
Iteration:  129
	Trajectories constructed.
	Epsilon: 0.00092728
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.955
	Step-size adapted to: 0.01909392
Iteration:  130
	Trajectories constructed.
	Epsilon: 0.00082996
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.948
	Step-size adapted to: 0.02733780
Iteration:  131
	Trajectories constructed.
	Epsilon: 0.00074238
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.936
	Step-size adapted to: 0.03891074
Iteration:  132
	Trajectories constructed.
	Epsilon: 0.00065512
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.921
	Step-size adapted to: 0.05496904
Iteration:  133
	Trajectories constr

	Trajectories constructed.
	Epsilon: 0.00000757
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.166
	Step-size adapted to: 0.26705915
Iteration:  171
	Trajectories constructed.
	Epsilon: 0.00000673
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.156
	Step-size adapted to: 0.25741001
Iteration:  172
	Trajectories constructed.
	Epsilon: 0.00000601
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.163
	Step-size adapted to: 0.24892962
Iteration:  173
	Trajectories constructed.
	Epsilon: 0.00000528
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.161
	Step-size adapted to: 0.24048800
Iteration:  174
	Trajectories constructed.
	Epsilon: 0.00000470
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.154
	Step-size adapted to: 0.23147461
Iteration:  175
	Trajectories constr

	Trajectories constructed.
	Epsilon: 0.00000007
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.133
	Step-size adapted to: 0.03907450
Iteration:  213
	Trajectories constructed.
	Epsilon: 0.00000006
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.126
	Step-size adapted to: 0.03708714
Iteration:  214
	Trajectories constructed.
	Epsilon: 0.00000006
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.148
	Step-size adapted to: 0.03559731
Iteration:  215
	Trajectories constructed.
	Epsilon: 0.00000005
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.134
	Step-size adapted to: 0.03392222
Iteration:  216
	Trajectories constructed.
	Epsilon: 0.00000005
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.126
	Step-size adapted to: 0.03220978
Iteration:  217
	Trajectories constr

	Trajectories constructed.
	Epsilon: 0.00000000
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.154
	Step-size adapted to: 0.00632601
Iteration:  255
	Trajectories constructed.
	Epsilon: 0.00000000
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.173
	Step-size adapted to: 0.00614888
Iteration:  256
	Trajectories constructed.
	Epsilon: 0.00000000
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.127
	Step-size adapted to: 0.00583965
Iteration:  257
	Trajectories constructed.
	Epsilon: 0.00000000
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.156
	Step-size adapted to: 0.00562809
Iteration:  258
	Trajectories constructed.
	Epsilon: 0.00000000
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.173
	Step-size adapted to: 0.00547050
Iteration:  259
	Trajectories constr

	Trajectories constructed.
	Epsilon: 0.00000000
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.167
	Step-size adapted to: 0.00131281
Iteration:  297
	Trajectories constructed.
	Epsilon: 0.00000000
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.171
	Step-size adapted to: 0.00127452
Iteration:  298
	Trajectories constructed.
	Epsilon: 0.00000000
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.170
	Step-size adapted to: 0.00123673
Iteration:  299
	Trajectories constructed.
	Epsilon: 0.00000000
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.152
	Step-size adapted to: 0.00118954
Iteration:  300
	Trajectories constructed.
	Epsilon: 0.00000000
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.144
	Step-size adapted to: 0.00113959
Iteration:  301
	Trajectories constr

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(16, 4))
# Tolerances
ax[0].plot(SMC_RWM.εs, label='SMC-RWM')
ax[0].plot(MS_RWM.εs, label='MS-RWM')
ax[0].plot(SMC_RWM_THEN_THUG.ϵs, label='SMC-RT')
ax[0].plot(MS_RWM_THEN_THUG.ϵs, label='MS-RT')
ax[0].axvline(x=SMC_RWM_THEN_THUG.n_switch-1, color='gray', linestyle='--')
ax[0].axvline(x=MS_RWM_THEN_THUG.n_switch-1, color='violet', linestyle='--')
ax[0].set_yscale('log')
ax[0].set_title("ε")
# Acceptance Probability
ax[1].plot(SMC_RWM.APS, label='SMC-RWM')
ax[1].plot(MS_RWM.PROP_MOVED, label='MS-RWM')
ax[1].plot(SMC_RWM_THEN_THUG.APS, label='SMC-RT')
ax[1].plot(MS_RWM_THEN_THUG.PROP_MOVED, label='MS-RT')
ax[1].axvline(x=SMC_RWM_THEN_THUG.n_switch-1, color='gray', linestyle='--')
ax[1].axvline(x=MS_RWM_THEN_THUG.n_switch-1, color='violet', linestyle='--')
ax[1].set_title("AP")
# Step size
ax[2].plot(SMC_RWM.δs, label='SMC-RWM')
ax[2].plot(MS_RWM.δs, label='MS-RWM')
ax[2].plot(SMC_RWM_THEN_THUG.δs, label='SMC-RT')
ax[2].plot(MS_RWM_THEN_THUG.δs, label='MS-RT')
ax[2].axvline(x=SMC_RWM_THEN_THUG.n_switch-1, color='gray', linestyle='--')
ax[2].axvline(x=MS_RWM_THEN_THUG.n_switch-1, color='violet', linestyle='--')
ax[2].set_yscale('log')
ax[2].set_title("δ")
# ESS
ax[3].plot(np.array(SMC_RWM.ESS) / SMC_RWM.N, label='SMC-RWM')
ax[3].plot(np.array(MS_RWM.ESS) / (MS_RWM.N*(MS_RWM.B+1)), label='MS-RWM')
ax[3].plot(np.array(SMC_RWM_THEN_THUG.ESS) / SMC_RWM_THEN_THUG.N, label='SMC-RT')
ax[3].plot(np.array(MS_RWM_THEN_THUG.ESS) / (MS_RWM_THEN_THUG.N*(MS_RWM_THEN_THUG.B+1)), label='MS-RT')
ax[3].axvline(x=SMC_RWM_THEN_THUG.n_switch-1, color='gray', linestyle='--')
ax[3].axvline(x=MS_RWM_THEN_THUG.n_switch-1, color='violet', linestyle='--')
ax[3].set_title("ESS")
ax[3].set_yscale('log')
plt.legend()
plt.show()

# THUG Only

#### SMC

#### Markov Snippets

# Histograms

In [ ]:
# def plot_histogram(n):
#     fig, ax = plt.subplots(figsize=(20, 4))
#     _, bins, _ = ax.hist(MS_RWM.K_RESAMPLED[n, :], density=True, bins=20, edgecolor='k', color='lightsalmon')
#     ax.set_xticks(bins)
#     ax.set_xticklabels(bins.astype(int))
#     return plt.show()
    
# interact(plot_histogram, n=(0, len(MS_RWM.ϵs) - 2))